# Création de lexique.txt pour le package french_preprocessing

## Imports

In [1]:
# -*- coding: utf-8 -*-
import nltk
from nltk.tag import StanfordPOSTagger
import csv
import os
import unicodedata

## Initialisation du StanfordPOSTagger

Il faut peut-être changer les path pour le StanfordPOSTagger et Java (JRE).

In [2]:
jar = './stanford-postagger-full-2018-10-16/stanford-postagger-3.9.2.jar'
model = './stanford-postagger-full-2018-10-16/models/french.tagger'
java_path = "C:/Program Files (x86)/Java/jre1.8.0_241/bin/java.exe"

os.environ['JAVAHOME'] = java_path
pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')

# Récupération du lexique du  FrenchLefffLemmatizer et modification des tags

#### Chargement du lexique : la LEFFF_TABLE

In [3]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()
lemmatizer.LEFFF_TABLE
tab = lemmatizer.LEFFF_TABLE

#### Liste des tags de la LEFFF_TABLE

In [4]:
# Donne la liste des tags de la LEFFF_TABLE
tag_list = []
tag_list_compt = [0 for i in range(100)]
for key in tab:
    for tag in tab[key].keys():
        if tag not in tag_list:
            tag_list.append(tag)
            ind = tag_list.index(tag)
            tag_list_compt[ind] += 1
        else:
            ind = tag_list.index(tag)
            tag_list_compt[ind] += 1

print(tag_list)

['nc', 'np', 'v', 'adj', 'parento', 'parentf', 'ponctw', 'adv', 'nc_2', 'pro', 'det', 'coo', 'auxAvoir', 'cld', 'clneg', 'clr', 'auxEtre', 'pri', 'prep', 'csu', 'etr', 'cln', 'prel', 'poncts', 'cll', 'pres', 'sbound', 'cla', 'advneg', 'clg', 'adjPref', 'advPref', ':GR', 'GA:', 'GP:', 'GN:', 'PV:', ':GA', 'NV:', ':PV', ':GN', ':GP', ':NV', 'GR:', 'ce', 'suffAdj', 'ilimp', 'clar', 'cldr', 'meta', 'epsilon', 'caimp', 'que', 'que_restr']


#### Catégorisation des différents tags pour l'uniformisation

Liste des tags uniformisés : ['v', 'nc', 'adj', 'c', 'npp', 'adv', 'det', 'pro', 'prep', 'i', 'ponct', 'cl', 'et']

In [5]:
# Tags à modifier en 'npp'
tag_list_np = ['np']
# Tags à modifier en 'cl'
tag_list_cl = ['cld','cla','clr','clneg','clg','cll','cln', 'cldr', 'clar']
# Tags à modifier en 'i'
tag_list_interjection = ['pres']
# Tags à modifier en 'pro'
tag_list_pr = ['prel']
# Tags à modifier en 'ponct'
tag_list_ponct = ['poncts','ponctw']
# Tags à modifier en 'adj'                
tag_list_adj = ['adjPref','suffAdj']
# Tags à modifier en 'adv'
tag_list_adv = ['advneg','advPref']
# Tags à modifier en 'c'
tag_list_cs = ['csu']

# Tags à supprimer
tag_list_ga = ['GA:',':GA']
tag_list_gp = ['GP:',':GP']               
tag_list_nv = [':NV','NV:']
tag_list_pv = ['PV:',':PV']
tag_list_gr = ['GR:',':GR']
tag_list_gn = [':GN','GN:']
tag_list_aut = ['sbound','epsilon','ilimp','caimp','meta', 'ce']
tag_list_parent = ['parentf','parento']
tag_list_aux = ['auxEtre','auxAvoir','etr']
tag_list_pro_int = ['pri']
tag_list_que = ['que_restr','que']
tag_list_coor = ['coo']

#### Suppression des tags de la LEFFF_TABLE non utilisés (et des lemmes associés)

ATTENTION : Cellule à executer deux fois pour la suppression, il y a un bug lors de la première execution.

In [7]:
for key in tab:
    if key == '_error':
        del tab[key]
    else :
        list_tag_supr = []
        for tag in tab[key].keys():
            if tag in ['GA:',':GA','GP:',':GP',':NV','NV:',
                       'PV:',':PV','GR:',':GR',':GN','GN:',
                       'sbound','epsilon','ilimp','caimp',
                       'meta', 'ce','parentf','parento',
                       'auxEtre','auxAvoir','etr','pri','coo',
                       'que_restr','que']:
                list_tag_supr.append(tag)
        for e in list_tag_supr:
            del tab[key][e]

#### Modification des tags restant de la LEFFF_TABLE

In [8]:
for key in tab:
    list_tag = []
    
    for tag in tab[key].keys():
        list_tag.append(tag)
        
    i=0
    while i>=0 and i<len(list_tag):
        i += 1
        for tag in tab[key].keys():
            
            if tag == 'np':
                nouveau_tag = 'npp'
                val = tab[key][tag]
                del tab[key][tag]
                tab[key][nouveau_tag] = val
                break
            
            elif tag == 'nc_2':
                nouveau_tag = 'nc'
                val = tab[key][tag]
                del tab[key][tag]
                tab[key][nouveau_tag] = val
                break

            elif tag in ['cld','cla','clr','clneg','clg',
                       'cll','cln', 'cldr', 'clar']:
                if 'cl' not in list_tag:
                    nouveau_tag = 'cl'
                    del tab[key][tag]
                    tab[key][nouveau_tag] = pos_tagger.tag([key])[0][0]
                    i -= 1
                else:
                    del tab[key][tag]
                    i -= 1
                break

            elif tag == 'pres':
                nouveau_tag = 'i'
                val = tab[key][tag]
                del tab[key][tag]
                tab[key][nouveau_tag] = val
                i -= 1
                break

            elif tag == 'prel':
                nouveau_tag = 'pro'
                val = tab[key][tag]
                del tab[key][tag]
                tab[key][nouveau_tag] = val
                i -= 1
                break

            elif tag in ['poncts','ponctw']:
                if 'ponct' not in list_tag:
                    nouveau_tag = 'ponct'
                    val = tab[key][tag]
                    del tab[key][tag]
                    i -= 1
                    tab[key][nouveau_tag] = val
                else:
                    del tab[key][tag]
                    i -= 1 
                break

            elif tag in ['adjPref','suffAdj']:
                if 'adj' not in list_tag:
                    nouveau_tag = 'adj'
                    val = tab[key][tag]
                    del tab[key][tag]
                    i -= 1
                    tab[key][nouveau_tag] = val
                else:
                    del tab[key][tag]
                    i -= 1 
                break

            elif tag in ['advneg','advPref']:
                if 'adv' not in list_tag:
                    nouveau_tag = 'adv'
                    val = tab[key][tag]
                    tab[key][nouveau_tag] = val
                    del tab[key][tag]
                    i -= 1
                else:
                    del tab[key][tag]
                    i -= 1 
                break


            elif tag in ['csu','coo']:
                if 'c' not in list_tag:
                    nouveau_tag = 'c'
                    val = tab[key][tag]
                    del tab[key][tag]
                    #i -= 1
                    tab[key][nouveau_tag] = val
                else:
                    del tab[key][tag]
                    i -= 1 
                break


#### Vérification de la modification effective des tags de la LEFFF_TABLE

In [9]:
tag_list = []
tag_list_compt = [0 for i in range(100)]
for key in tab:
    for tag in tab[key].keys():
        if tag not in tag_list:
            tag_list.append(tag)
            ind = tag_list.index(tag)
            tag_list_compt[ind] += 1
        else:
            ind = tag_list.index(tag)
            tag_list_compt[ind] += 1

for i in range(len(tag_list)):
    print(tag_list[i],tag_list_compt[i])

nc 86234
npp 52283
v 301207
adj 57217
ponct 46
adv 3820
pro 149
det 178
prep 733
c 324
i 93
cl 62


# Fonctions de réduction des tags pour les autres lexiques

#### Réduction des tags du StanfordPOSTagger pour correspondre aux tags : ['v', 'nc', 'adj', 'c', 'npp', 'adv', 'det', 'pro', 'prep', 'i', 'ponct', 'cl', 'et']

In [10]:
final_list = ['v', 'nc', 'adj', 'c', 'npp', 'adv', 'det', 'pro', 'prep', 'i', 'ponct', 'cl', 'et']

def reduction_tag_stanford(tag):
    tag_reduc = ''
    if tag in ['VS', 'VINF', 'VPP', 'VPR', 'VIMP']:
        tag_reduc = 'V'
    elif tag in ['N','PUNC','PREF']:
        tag_reduc = 'NC'
    elif tag in ['CS', 'CC']:
        tag_reduc = 'C'
    elif tag in ['CLS', 'CLO', 'CLR']:
        tag_reduc = 'CL'
    elif tag in ['ADJWH', 'A']:
        tag_reduc = 'ADJ'
    elif tag == 'ADVWH':
        tag_reduc = 'ADV'
    elif tag in ['PROREL', 'PROWH']:
        tag_reduc = 'PRO'
    elif tag in ['DETWH']:
        tag_reduc = 'DET'
    else:
        tag_reduc = tag
    return(tag_reduc.lower())

#### Réduction des tags du Lexique383 pour correspondre aux tags : ['v', 'nc', 'adj', 'c', 'npp', 'adv', 'det', 'pro', 'prep', 'i', 'ponct', 'cl', 'et']

In [11]:
#Liste des tags originels de lexique383 : ['ADV', 'ADJ','PRO', 'PRE', 'NOM', 'VER', 'ONO', 'CON', 'AUX', 'ART', 'EXP']

def reduction_tag_lexique383(tag):
    nouveau_tag = ''
    if tag == 'NOM':
        nouveau_tag = 'nc'
    elif tag == 'VER' or tag == 'AUX':
        nouveau_tag = 'v'
    elif tag == 'CON':
        nouveau_tag = 'c'
    elif tag == 'ONO':
        nouveau_tag = 'i'
    elif tag == 'ART':
        nouveau_tag = 'det'
    elif tag == 'PRE':
        nouveau_tag = 'prep'
    elif tag == 'EXP':
        nouveau_tag = 'cl'
    else:
        nouveau_tag = tag.lower()
    return nouveau_tag
        

# Création du lexique final

On regroupe les éléments des trois lexiques : 

- le lexique de la LEFFF_TABLE
- lexique383_noLEFFF (préalablement créé)
- lexique_spacy_noLEFFF_noLexique383 (préalablement créé)

### Chargement de lexique383_noLEFFF

In [12]:
lexique383_noLEFFF = []

with open('./intermediate_data/lexique383_noLEFFF.txt') as f:
        read = csv.reader(f)
        for row in read:
            lexique383_noLEFFF.append(row)

### Uniformisation des tags de lexique383_noLEFFF

In [13]:
for row in lexique383_noLEFFF:
    tag_lexique383 = row[2]
    row[2] = reduction_tag_lexique383(tag_lexique383)

### Chargement de lexique_spacy_noLEFFF_noLexique383

In [14]:
lexique_spacy_noLEFFF_noLexique383 = []

with open('./intermediate_data/lexique_spacy_noLEFFF_noLexique383.txt') as f:
        read = csv.reader(f)
        for row in read:
            lexique_spacy_noLEFFF_noLexique383.append(row)

### Uniformisation des tags de lexique_spacy_noLEFFF_noLexique383

In [15]:
for row in lexique_spacy_noLEFFF_noLexique383:
    tag = row[2]
    row[2] = reduction_tag_stanford(tag)

### Ajouts des nouveaux mots dans la LEFFF_TABLE

#### Ajout des mots du fichier lexique383_noLEFFF.txt dont les tags ont été réduits

In [16]:
for row in lexique383_noLEFFF:
    tab[row[0]] = {row[2]:row[1]}

#### Ajout des mots du fichier lexique_spacy_noLEFFF_noLexique383.txt dont les tags ont été réduits

In [17]:
for row in lexique_spacy_noLEFFF_noLexique383:
    tab[row[0]] = {row[2]:row[1]}

### Transformation des majuscules des mots du lexique en minuscules

In [18]:
lexique = {}

for key in tab:
    new_key = key.lower()
    if new_key != key:
        lexique[new_key] = tab[key] 
    else:
        lexique[key] = tab[key]

### Ajout de quelques lemmes manquants

In [1]:
dic = {"n'":{'cl': "n'", 'adv': 'ne'},
       "$":{'nc': 'dollar', 'adj': 'dollar', 'adv': 'dollar', 'det': 'dollar'},
       "£":{'nc': 'pound', 'adj': 'pound', 'adv': 'pound', 'det': 'pound'},
       '€':{'nc': 'euro', 'adj': 'euro', 'adv': 'euro', 'det': 'euro'},
       '%':{'nc': 'pourcent', 'adj': 'pourcent', 'det': 'pourcent'},
       "j'":{'cl': 'je', 'pro': 'je'},
       "appelle":{'v': 'appeler', 'npp': 'Appelle'},
       "&":{'nc': 'et', 'c': 'et', 'adv': 'et'}}

for key in dic:
    if key not in lexique.keys():
        lexique[key]=dic[key]
    else:
        del lexique[key]
        lexique[key]=dic[key]

### Ecriture du lexique dans le fichier lexique.txt

In [19]:
with open('./final_data/lexique.txt', 'w') as f:
    f.write(str(lexique))